# Audio Feature Extraction

## 1. Importing Library

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import librosa

## 2. Define function for extraction

In [2]:
def get_features(path, duration):

    signal, sr = librosa.load(path, sr=22050, duration=duration)

    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
    delta_mfcc = librosa.feature.delta(mfcc, mode='mirror')
    delta2_mfcc = librosa.feature.delta(mfcc, mode='mirror', order=2)

    mfccs = np.hstack((
                    np.mean(mfcc, axis=1),
                    np.std(mfcc, axis=1),
                    skew(mfcc, axis=1),
                    kurtosis(mfcc, axis=1)
                ))
    delta_mfccs = np.hstack((
                    np.mean(delta_mfcc, axis=1),
                    np.std(delta_mfcc, axis=1),
                    skew(delta_mfcc, axis=1),
                    kurtosis(delta_mfcc, axis=1)
                ))
    delta2_mfccs = np.hstack((
                    np.mean(delta2_mfcc, axis=1),
                    np.std(delta2_mfcc, axis=1),
                    skew(delta2_mfcc, axis=1),
                    kurtosis(delta2_mfcc, axis=1)
                ))

    extracted_features = np.hstack((mfccs, delta_mfccs, delta2_mfccs))
    return extracted_features

## 3. Get Audio Features for GPT Audio Data

In [8]:
# dataset_path = '../data/guitar/wav'
# duration = 4.0

dataset_path = '../data/guitar/wav_split'
duration = 0.3

data = []
for i, (root, _, files) in enumerate(os.walk(dataset_path)):
    if root is not dataset_path:
        if not files:
            tone_type = root.split('/')[-1]
        else:
            subtechnique = root.split('/')[-1]
            technique = subtechnique.split('_')[0]
            for file in files:
                file_path = os.path.join(root, file)
                features = get_features(file_path, duration)
                features = np.append(features, [technique, subtechnique, tone_type, file_path])
                data.append(features)
print('Done!!!')

Done!!!


## 4. Define Columns Name to Create Dataframe

In [9]:
columns = []
names = ['mfcc', 'delta_mfcc', 'delta2_mfcc']
stats = ['mean', 'std', 'skew', 'kurtosis']
file_desc = ['technique', 'subtechnique', 'tone_type', 'file_path']

for name in names:
    for stat in stats:
        for i in range(13):
            col = f'{stat}_{name}_{i+1}'
            columns = np.append(columns, col)

columns = np.append(columns, file_desc)
columns.shape

(160,)

## 5. Convert Extracted Feature (List) to Dataframe

In [10]:
df = pd.DataFrame(data, columns=columns)
print(df.shape)

(7657, 160)


## 6. Save Dataframe to CSV File

In [11]:
# df.to_csv('../data/gpt_complete.csv', index=False)
df.to_csv('../data/gpt_split.csv', index=False)

## 7. Load CSV File

In [12]:
# gpt = pd.read_csv('../data/gpt_complete.csv')
gpt = pd.read_csv('../data/gpt_split.csv')
gpt

,mean_mfcc_1,mean_mfcc_2,mean_mfcc_3,mean_mfcc_4,mean_mfcc_5,mean_mfcc_6,mean_mfcc_7,mean_mfcc_8,mean_mfcc_9,mean_mfcc_10,...,kurtosis_delta2_mfcc_8,kurtosis_delta2_mfcc_9,kurtosis_delta2_mfcc_10,kurtosis_delta2_mfcc_11,kurtosis_delta2_mfcc_12,kurtosis_delta2_mfcc_13,technique,subtechnique,tone_type,file_path
0,-321.30880,53.664543,-41.657486,20.575792,-34.078175,-15.377439,11.446047,15.791092,24.217379,17.853090,...,-0.121737,-0.032582,-0.651918,0.070048,-0.568145,-0.889111,normal,normal_whole_step_up,5,../data/guitar/wav_split/5/normal_whole_step_u...
1,-197.10857,86.117830,-43.673218,54.265663,-19.434647,-6.111839,-9.582001,-11.827796,2.867603,4.243337,...,-0.689194,-1.064827,-0.658298,-0.828307,-0.833295,-0.063402,normal,normal_whole_step_up,5,../data/guitar/wav_split/5/normal_whole_step_u...
2,-244.01527,82.072920,-51.395523,53.868187,-19.187164,-2.757237,-3.269154,-6.843103,-5.374433,-7.197699,...,-0.365610,0.674864,-1.053009,-1.213758,-0.678068,-0.446414,normal,normal_whole_step_up,5,../data/guitar/wav_split/5/normal_whole_step_u...
3,-194.70702,95.650440,-42.411100,60.186470,-19.122845,-0.457996,-7.523692,-14.381029,-5.291275,-6.963178,...,-0.686471,-1.336491,-0.994807,-1.310098,-1.498797,-1.144028,normal,normal_whole_step_up,5,../data/guitar/wav_split/5/normal_whole_step_u...
4,-346.38394,64.545020,-35.303170,33.381020,-25.183414,-16.508846,-1.485640,5.415703,14.938007,19.732126,...,0.516332,1.466204,-0.063096,-0.961692,-1.165767,-0.681691,normal,normal_whole_step_up,5,../data/guitar/wav_split/5/normal_whole_step_u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7652,-208.62580,89.534290,-72.583420,49.182014,-24.961325,-16.623670,-23.536650,-6.877629,-6.587377,13.314508,...,-0.441189,-0.830679,-0.824871,-1.261925,0.522229,1.186630,slide,slide_whole_step_down,4,../data/guitar/wav_split/4/slide_whole_step_do...
7653,-171.62112,102.377240,-76.012450,47.581974,-17.870640,-2.976126,-21.729235,-0.982900,-12.493032,-15.892898,...,-1.347501,-1.361435,-0.552934,-1.073264,-1.005519,0.375271,slide,slide_whole_step_down,4,../data/guitar/wav_split/4/slide_whole_step_do...
7654,-180.56640,105.330930,-84.613040,53.629623,-20.374622,-4.217498,-24.346489,-6.077085,-16.929007,-11.143519,...,-0.885467,0.442155,-0.373782,-0.035077,-1.147394,-0.587127,slide,slide_whole_step_down,4,../data/guitar/wav_split/4/slide_whole_step_do...
7655,-186.92618,102.576010,-76.464424,46.970966,-22.491465,-5.077390,-29.038008,1.667840,-8.605193,-7.224083,...,-1.039444,-1.278862,-1.104712,-1.181983,-0.656139,-0.785504,slide,slide_whole_step_down,4,../data/guitar/wav_split/4/slide_whole_step_do...
